In [1]:
import torch

In [2]:
import tensorflow as tf
import transformers
from transformers import BertTokenizer, BertForSequenceClassification, AdamW


In [3]:
import gensim
from gensim.models import Word2Vec
from nltk.stem import WordNetLemmatizer
import tensorflow_hub as hub
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno
import time
%matplotlib inline
import os
from os import listdir
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, auc, roc_auc_score, roc_curve, classification_report


In [4]:
import tensorflow_hub as hub

In [5]:
df = pd.read_csv('Source/flipkart_com-ecommerce_sample_1050.csv',sep=',')

In [6]:
df.head(3)

,uniq_id,crawl_timestamp,product_url,product_name,product_category_tree,pid,retail_price,discounted_price,image,is_FK_Advantage_product,description,product_rating,overall_rating,brand,product_specifications
0,55b85ea15a1536d46b7190ad6fff8ce7,2016-04-30 03:22:56 +0000,http://www.flipkart.com/elegance-polyester-mul...,Elegance Polyester Multicolor Abstract Eyelet ...,"[""Home Furnishing >> Curtains & Accessories >>...",CRNEG7BKMFFYHQ8Z,1899.0,899.0,55b85ea15a1536d46b7190ad6fff8ce7.jpg,False,Key Features of Elegance Polyester Multicolor ...,No rating available,No rating available,Elegance,"{""product_specification""=>[{""key""=>""Brand"", ""v..."
1,7b72c92c2f6c40268628ec5f14c6d590,2016-04-30 03:22:56 +0000,http://www.flipkart.com/sathiyas-cotton-bath-t...,Sathiyas Cotton Bath Towel,"[""Baby Care >> Baby Bath & Skin >> Baby Bath T...",BTWEGFZHGBXPHZUH,600.0,449.0,7b72c92c2f6c40268628ec5f14c6d590.jpg,False,Specifications of Sathiyas Cotton Bath Towel (...,No rating available,No rating available,Sathiyas,"{""product_specification""=>[{""key""=>""Machine Wa..."
2,64d5d4a258243731dc7bbb1eef49ad74,2016-04-30 03:22:56 +0000,http://www.flipkart.com/eurospa-cotton-terry-f...,Eurospa Cotton Terry Face Towel Set,"[""Baby Care >> Baby Bath & Skin >> Baby Bath T...",BTWEG6SHXTDB2A2Y,NaN,NaN,64d5d4a258243731dc7bbb1eef49ad74.jpg,False,Key Features of Eurospa Cotton Terry Face Towe...,No rating available,No rating available,Eurospa,"{""product_specification""=>[{""key""=>""Material"",..."


In [7]:
# premier notebook NLP 40 % 
# deux parties : texte (données non structurées)
# deuxième notebook image 40 % 

cols = ['product_name',
        'product_category_tree',
        'description',
        'brand']

df_texte = df.loc[:,cols]
df_texte

,product_name,product_category_tree,description,brand
0,Elegance Polyester Multicolor Abstract Eyelet ...,"[""Home Furnishing >> Curtains & Accessories >>...",Key Features of Elegance Polyester Multicolor ...,Elegance
1,Sathiyas Cotton Bath Towel,"[""Baby Care >> Baby Bath & Skin >> Baby Bath T...",Specifications of Sathiyas Cotton Bath Towel (...,Sathiyas
2,Eurospa Cotton Terry Face Towel Set,"[""Baby Care >> Baby Bath & Skin >> Baby Bath T...",Key Features of Eurospa Cotton Terry Face Towe...,Eurospa
3,SANTOSH ROYAL FASHION Cotton Printed King size...,"[""Home Furnishing >> Bed Linen >> Bedsheets >>...",Key Features of SANTOSH ROYAL FASHION Cotton P...,SANTOSH ROYAL FASHION
4,Jaipur Print Cotton Floral King sized Double B...,"[""Home Furnishing >> Bed Linen >> Bedsheets >>...",Key Features of Jaipur Print Cotton Floral Kin...,Jaipur Print
...,...,...,...,...
1045,Oren Empower Extra Large Self Adhesive Sticker,"[""Baby Care >> Baby & Kids Gifts >> Stickers >...",Oren Empower Extra Large Self Adhesive Sticker...,Oren Empower
1046,Wallmantra Large Vinyl Sticker Sticker,"[""Baby Care >> Baby & Kids Gifts >> Stickers >...",Wallmantra Large Vinyl Sticker Sticker (Pack o...,Wallmantra
1047,Uberlyfe Extra Large Pigmented Polyvinyl Films...,"[""Baby Care >> Baby & Kids Gifts >> Stickers >...",Buy Uberlyfe Extra Large Pigmented Polyvinyl F...,Uberlyfe
1048,Wallmantra Medium Vinyl Sticker Sticker,"[""Baby Care >> Baby & Kids Gifts >> Stickers >...",Buy Wallmantra Medium Vinyl Sticker Sticker fo...,Wallmantra


In [8]:
# lire des caractère sur une image
import nltk
nltk.download('punkt')
tokenizer = nltk.RegexpTokenizer(r'\w+')
token = np.array(df_texte.product_category_tree)

token = [tokenizer.tokenize(i.lower()) for i in token]

bigram = [list(nltk.bigrams(bigram)) for bigram in token]

token_bigram = pd.Series(bigram)

cat_name = np.array([token_bigram[i][0] for i in range(len(token_bigram))])

    

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Dragomir\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [9]:
df_token = pd.DataFrame(cat_name, columns=['cat1','cat2'])
df_token = df_token.cat1 + ' ' +  df_token.cat2

In [10]:
df_texte['category_name'] = df_token

In [11]:
df_texte.loc[df_texte.category_name == 'computers laptops','category_name'] = 'computers laptop'

In [12]:
token2 = np.array(df_texte.description)
token2 = [tokenizer.tokenize(i.lower()) for i in token2]

In [13]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Dragomir\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [14]:
sw = set()
sw.update(tuple(nltk.corpus.stopwords.words('english')))

In [15]:
descript_sw = [[idx for idx in token if idx not in sw] for token in token2]

In [16]:
from nltk.stem.snowball import EnglishStemmer

stemmer = EnglishStemmer()

In [17]:
data_stem = [stemmer.stem(str(i)) for i in descript_sw]

data_stem[0][:100]

"['key', 'features', 'elegance', 'polyester', 'multicolor', 'abstract', 'eyelet', 'door', 'curtain', "

In [18]:
Stem_word = [''.join(stemword) for stemword in data_stem]
Stem_word[:2]

["['key', 'features', 'elegance', 'polyester', 'multicolor', 'abstract', 'eyelet', 'door', 'curtain', 'floral', 'curtain', 'elegance', 'polyester', 'multicolor', 'abstract', 'eyelet', 'door', 'curtain', '213', 'cm', 'height', 'pack', '2', 'price', 'rs', '899', 'curtain', 'enhances', 'look', 'interiors', 'curtain', 'made', '100', 'high', 'quality', 'polyester', 'fabric', 'features', 'eyelet', 'style', 'stitch', 'metal', 'ring', 'makes', 'room', 'environment', 'romantic', 'loving', 'curtain', 'ant', 'wrinkle', 'anti', 'shrinkage', 'elegant', 'apparance', 'give', 'home', 'bright', 'modernistic', 'appeal', 'designs', 'surreal', 'attention', 'sure', 'steal', 'hearts', 'contemporary', 'eyelet', 'valance', 'curtains', 'slide', 'smoothly', 'draw', 'apart', 'first', 'thing', 'morning', 'welcome', 'bright', 'sun', 'rays', 'want', 'wish', 'good', 'morning', 'whole', 'world', 'draw', 'close', 'evening', 'create', 'special', 'moments', 'joyous', 'beauty', 'given', 'soothing', 'prints', 'bring', 'ho

### Word embeding

In [19]:
import gensim
from gensim.models import Word2Vec

In [20]:
from sklearn import preprocessing


# preprocessing encodage des données
encodage = preprocessing.LabelEncoder()
encodage.fit(df_texte.category_name)
df_texte["label_cat1"] = encodage.transform(df_texte.category_name)
# data.head(5)
df_texte.category_name.value_counts()

home furnishing       150
baby care             150
home decor            150
kitchen dining        150
beauty and            150
watches wrist         149
computers laptop       89
computers network      49
computers computer      6
computers tablet        3
computers storage       2
computers software      1
watches clocks          1
Name: category_name, dtype: int64

In [21]:
Stem_word2 = [tokenizer.tokenize(i.lower()) for i in Stem_word]

In [22]:
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()

Lemmatise_corpus = [[lemmatizer.lemmatize(Lemma) for Lemma in Stem_word2[idx]] for idx in range(len(Stem_word2))]

Lemmatise_corpus[0][10]
                                        

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Dragomir\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Dragomir\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


'curtain'

In [23]:
model = Word2Vec(Lemmatise_corpus, min_count=1,hs=1,negative=0,sg=0,epochs=5)  # min_count spécifie la fréquence minimale d'un mot pour être inclus dans le modèle

In [24]:
model.wv.most_similar('home')

[('gifting', 0.7993993163108826),
 ('crafted', 0.794390857219696),
 ('look', 0.7491447329521179),
 ('express', 0.7399798631668091),
 ('one', 0.7333756685256958),
 ('permanat', 0.7186675071716309),
 ('pic', 0.716445803642273),
 ('wood', 0.7043812274932861),
 ('gift', 0.6971081495285034),
 ('help', 0.693030834197998)]

In [25]:
model.wv.most_similar('home')

[('gifting', 0.7993993163108826),
 ('crafted', 0.794390857219696),
 ('look', 0.7491447329521179),
 ('express', 0.7399798631668091),
 ('one', 0.7333756685256958),
 ('permanat', 0.7186675071716309),
 ('pic', 0.716445803642273),
 ('wood', 0.7043812274932861),
 ('gift', 0.6971081495285034),
 ('help', 0.693030834197998)]

In [26]:
category_name = [tokenizer.tokenize(i) for i in df_texte.category_name.unique()]
category_name

[['home', 'furnishing'],
 ['baby', 'care'],
 ['watches', 'wrist'],
 ['home', 'decor'],
 ['kitchen', 'dining'],
 ['beauty', 'and'],
 ['computers', 'network'],
 ['computers', 'tablet'],
 ['computers', 'laptop'],
 ['computers', 'software'],
 ['computers', 'computer'],
 ['computers', 'storage'],
 ['watches', 'clocks']]

In [27]:
documents = Lemmatise_corpus

# Initialize an empty array to store document vectors
document_vectors = np.zeros((len(documents), model.vector_size))
tab = []
save = [tab.append(len(doc)) for doc in documents]


In [28]:
# récupére tous les vecteurs 
def wvector(corpus,model):
    # List of documents
    documents = corpus

    # Initialize an empty array to store document vectors
    document_vectors = np.zeros((len(documents), model.vector_size))
    # Iterate over each document

    for i, doc in enumerate(documents):
        # Split the document into individual words
        words = doc
        # Initialize an empty array to store word vectors
        word_vectors = np.zeros((len(words), model.vector_size))

        #document_vectors = np.zeros((len(documents), model.vector_size))

        # Iterate over each word in the document
        for j, word in enumerate(words):
            # Check if the word is in the vocabulary of the model
            if word in model.wv.index_to_key:
                # Get the word vector
                word_vectors[j] = model.wv[word]
        # Calculate the average vector of the document
        if len(word_vectors) > 0:
            #print(word_vectors.shape)
            document_vectors[i] = np.mean(word_vectors, axis=0)
    return document_vectors
        
    

In [29]:
from sklearn.pipeline import Pipeline 
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, BaggingClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split, GridSearchCV


In [34]:
def create_pipeline_evaluate_model(index_fold, X_train, X_test, y_train, y_test,idx_train,idx_test):
    
        model = Pipeline([
                         ("classifier",SVC())
                         ])

        model.fit(X_train,y_train)
        y_pred = model.predict(X_test)
        score = model.score(X_test,y_test)     
        print(f"Run {index_fold} : score = {round(score,2)} ")
        return (score)

In [35]:
nb_model = 5
index_fold = 0
kf = KFold(n_splits=nb_model, shuffle=False)
average_score = 0

X = document_vectors
y = df_texte.label_cat1

kf = KFold(n_splits=nb_model)

for train_index, test_index in kf.split(X):
    
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    score = create_pipeline_evaluate_model(index_fold, X_train, X_test, y_train, y_test,train_index,test_index)    
    average_score += score
    index_fold += 1
    
average_score = average_score / nb_model
print(average_score)

Run 0 : score = 0.09 
Run 1 : score = 0.01 
Run 2 : score = 0.04 
Run 3 : score = 0.07 
Run 4 : score = 0.0 
0.04095238095238095


In [36]:

y = df_texte.label_cat1

X_train, X_test, y_train, y_test = train_test_split(document_vectors, y, test_size=0.2, random_state=42)

model = Pipeline([
                 ("classifier",SVC())
                 ])

model.fit(X_train,y_train)
y_pred = model.predict(X_test)

score = model.score(X_test,y_test)     
print(f"Run {index_fold} : score = {round(score,2)} ")



Run 5 : score = 0.1 


In [37]:
pipe = Pipeline([
                ("estimator",LogisticRegression())
                ])
pipe


# Vectorise les mots avec la méthode continu bag of words
# min_count spécifie la fréquence minimale d'un mot pour être inclus dans le modèle
# epochs signifie que le réseau de neuronne va s'entrainer un nombre de fois sur les données 
model = Word2Vec(Lemmatise_corpus, min_count=1,hs=1,negative=0,sg=0,epochs=5)  

# retoune les vecteurs de mots par document
vectors = wvector(Lemmatise_corpus,model)
X = vectors
y = df_texte.label_cat1


param_grid = {
              'estimator':[RandomForestClassifier(),
                           AdaBoostClassifier(),
                           GradientBoostingClassifier(),
                           ExtraTreesClassifier(),
                           DecisionTreeClassifier(),
                           SVC(),
                           BaggingClassifier(),
                           KNeighborsClassifier(13),
                           LogisticRegression()
                          ]}


grid = GridSearchCV(pipe,param_grid=param_grid, cv=10,return_train_score=True ,n_jobs=-1, verbose=1)

grid.fit(X,y)

grid_cv = pd.DataFrame(grid.cv_results_)

new_cols = [i for i in grid_cv.columns if 'split' not in i.lower()]

grid_cv.loc[:,new_cols].sort_values('mean_test_score', ascending=False)



Fitting 10 folds for each of 9 candidates, totalling 90 fits


C:\Users\Dragomir\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:676: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  warnings.warn(


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_estimator,params,mean_test_score,std_test_score,rank_test_score,mean_train_score,std_train_score
3,0.658081,0.111044,0.040453,0.014389,ExtraTreesClassifier(),{'estimator': ExtraTreesClassifier()},0.837143,0.094612,1,1.000000,0.000000
5,0.106299,0.020809,0.034401,0.011316,SVC(),{'estimator': SVC()},0.826667,0.087991,2,0.873333,0.005334
0,1.255159,0.177889,0.025101,0.005320,RandomForestClassifier(),{'estimator': RandomForestClassifier()},0.825714,0.102402,3,1.000000,0.000000
8,0.194197,0.025820,0.001000,0.001095,LogisticRegression(),{'estimator': LogisticRegression()},0.819048,0.097497,4,0.858519,0.007719
2,55.622946,1.858243,0.004004,0.000770,GradientBoostingClassifier(),{'estimator': GradientBoostingClassifier()},0.799048,0.100831,5,1.000000,0.000000
6,0.913551,0.079286,0.004001,0.000448,BaggingClassifier(),{'estimator': BaggingClassifier()},0.795238,0.102685,6,0.993333,0.002119
7,0.003999,0.005040,0.012799,0.004285,KNeighborsClassifier(n_neighbors=13),{'estimator': KNeighborsClassifier(n_neighbors...,0.791429,0.105749,7,0.855238,0.005535
4,0.165499,0.023205,0.000700,0.000458,DecisionTreeClassifier(),{'estimator': DecisionTreeClassifier()},0.705714,0.119063,8,1.000000,0.000000
1,1.370283,0.099505,0.021800,0.004019,AdaBoostClassifier(),{'estimator': AdaBoostClassifier()},0.307619,0.024112,9,0.300317,0.008762


In [34]:


pipe = Pipeline([
                ("estimator",LogisticRegression())
                ])


# sg = 1 signifie que le modèle va vectorise les mots avec la méthode skip-gramms
# min_count spécifie la fréquence minimale d'un mot pour être inclus dans le modèle
# epochs signifie que le réseau de neuronne va s'entrainer un nombre de fois sur les données 
model = Word2Vec(Lemmatise_corpus, min_count=1,hs=1,negative=0,sg=1,epochs=5)  
vectors = wvector(Lemmatise_corpus,model)

X = vectors
y = df_texte.label_cat1

param_grid = {
              'estimator':[RandomForestClassifier(),
                           AdaBoostClassifier(),
                           GradientBoostingClassifier(),
                           ExtraTreesClassifier(),
                           DecisionTreeClassifier(),
                           SVC(),
                           BaggingClassifier(),
                           KNeighborsClassifier(13),
                           LogisticRegression()
                          ]}


grid = GridSearchCV(pipe,param_grid=param_grid, cv=10,return_train_score=True ,n_jobs=-1, verbose=1)

grid.fit(X,y)

grid_cv = pd.DataFrame(grid.cv_results_)

new_cols = [i for i in grid_cv.columns if 'split' not in i.lower()]

grid_cv.loc[:,new_cols].sort_values('mean_test_score', ascending=False)



Fitting 10 folds for each of 9 candidates, totalling 90 fits


C:\Users\Dragomir\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:676: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  warnings.warn(


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_estimator,params,mean_test_score,std_test_score,rank_test_score,mean_train_score,std_train_score
3,0.538190,0.112658,0.029913,0.013629,ExtraTreesClassifier(),{'estimator': ExtraTreesClassifier()},0.879048,0.078078,1,1.000000,0.000000
8,0.137698,0.010432,0.000800,0.000600,LogisticRegression(),{'estimator': LogisticRegression()},0.878095,0.079317,2,0.904444,0.006043
5,0.108599,0.018558,0.033000,0.012578,SVC(),{'estimator': SVC()},0.873333,0.088939,3,0.924339,0.005684
0,1.078608,0.138692,0.020801,0.001165,RandomForestClassifier(),{'estimator': RandomForestClassifier()},0.865714,0.084698,4,1.000000,0.000000
7,0.002199,0.000601,0.011598,0.004005,KNeighborsClassifier(n_neighbors=13),{'estimator': KNeighborsClassifier(n_neighbors...,0.829524,0.107619,5,0.904233,0.005036
2,57.575393,1.448996,0.004202,0.000976,GradientBoostingClassifier(),{'estimator': GradientBoostingClassifier()},0.823810,0.083598,6,1.000000,0.000000
6,0.959500,0.087200,0.004201,0.001990,BaggingClassifier(),{'estimator': BaggingClassifier()},0.817143,0.083658,7,0.997566,0.001500
4,0.198097,0.042441,0.001201,0.000400,DecisionTreeClassifier(),{'estimator': DecisionTreeClassifier()},0.756190,0.091765,8,1.000000,0.000000
1,1.273403,0.207514,0.025100,0.012293,AdaBoostClassifier(),{'estimator': AdaBoostClassifier()},0.318095,0.039544,9,0.321376,0.018650


In [35]:

from sklearn.metrics import accuracy_score, precision_recall_fscore_support as score
logistic = LogisticRegression()

model = Word2Vec(Lemmatise_corpus, min_count=1,hs=1,negative=0,sg=1,epochs=5)  
vectors = wvector(Lemmatise_corpus,model)

X = vectors
y = df_texte.label_cat1

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)
logistic.fit(X_train, y_train)



LogisticRegression()

In [36]:
y_pred = logistic.predict(X_test)
y_pred

array([12, 12,  3,  4,  0,  1,  1, 12,  8,  9,  4,  8,  8,  9,  3, 12,  9,
        1,  0,  9, 12, 10,  3,  3,  1,  8,  1, 10, 10,  8,  0, 12,  9,  8,
        3, 10, 12, 10,  3, 12,  9, 10, 12,  8,  1,  1,  9,  4,  1, 12, 12,
        8, 10,  1,  9,  9,  0,  9,  9,  3,  8,  8,  1, 10,  3,  4,  3,  0,
        4,  0, 10,  8,  8,  4,  0,  9,  9,  0,  8,  9, 12, 10,  9,  9,  9,
        9, 12,  8,  9,  8, 10,  9,  4,  8, 10,  8, 12,  9,  9, 10, 12,  9,
        8,  1,  1,  8,  0,  1, 10,  8,  1, 10, 12, 12,  9, 12,  8,  9,  9,
        0, 12,  3,  9, 10,  9, 10,  0,  3,  8, 12,  8,  9,  9,  4,  1,  8,
        3,  3, 12,  0,  1,  1,  1,  9, 10,  8, 10,  1,  1, 10,  1,  8,  4,
       12,  8,  9,  1, 10,  3,  9,  8, 12,  9,  3,  1,  9,  0,  9,  0,  9,
        9,  9,  1, 12,  8, 10,  4,  8, 12, 10,  8,  0,  1,  9, 12,  9, 10,
        1,  4,  1,  8,  0, 10,  9,  1,  9, 12,  8,  3,  1,  1, 10,  3,  8,
        3, 12, 12, 10,  1,  1])

In [37]:
np.array(y_test)

array([12, 12,  3,  4,  0,  1,  1, 12,  8,  9,  4,  8,  8,  9,  3, 12,  9,
        1,  0,  9, 12, 10,  3,  3,  1,  0,  1, 10, 10,  8,  0, 12,  9,  8,
        3,  0, 12, 10,  3, 12,  9, 10, 12,  8,  1,  1,  9,  4,  1, 12, 12,
        8, 10,  1,  9,  9,  0,  9,  9,  3,  8,  8,  1, 10,  3,  4,  3,  0,
        4,  9, 10,  8,  0,  4,  0,  9,  0,  0, 10,  9, 12, 10,  9,  9,  9,
        9, 12,  0,  9,  8, 10,  9,  4,  8,  0,  8, 12,  9,  9, 10, 12,  9,
        8,  1,  0,  8,  0, 10, 10,  8,  1, 10, 12, 12,  9, 12,  8,  9,  9,
        0, 12,  3,  9, 10,  0, 10,  0,  7,  8, 12,  8,  9,  9,  4,  1,  8,
        3,  3, 12,  0,  1,  1,  1,  9, 10,  8, 10,  1,  1, 10,  4,  8,  4,
       12,  0,  9,  1, 10,  3,  0,  8, 12,  9,  3,  1,  9,  8,  9,  1,  9,
        0,  9,  1, 12,  8, 10,  4,  8, 12,  6,  8,  0,  1,  9, 12,  9,  0,
        1,  8,  1,  8,  0, 10,  9,  1,  0, 12,  8,  3,  1,  0, 10,  3,  8,
        3, 12, 12, 10,  1,  1])

In [38]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.81      0.48      0.60        27
           1       0.88      0.97      0.92        29
           3       0.94      1.00      0.97        17
           4       0.91      0.91      0.91        11
           6       0.00      0.00      0.00         1
           7       0.00      0.00      0.00         1
           8       0.85      0.94      0.89        31
           9       0.88      0.97      0.93        39
          10       0.85      0.92      0.88        25
          12       1.00      1.00      1.00        29

    accuracy                           0.89       210
   macro avg       0.71      0.72      0.71       210
weighted avg       0.88      0.89      0.88       210



C:\Users\Dragomir\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Dragomir\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Dragomir\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### Classification de texte avec BERT


In [38]:
tokens = []
_ = [tokens.append(str(txt).replace("'", '').replace(",","").replace("[",'').replace("]",'')) for txt in Lemmatise_corpus]
print('Max sentence length: ', max([len(sen) for sen in tokens]))


Max sentence length:  2620


In [39]:
# Prétraitement des textes et des étiquettes
# utiliser la CPU 
device = torch.device("cpu" if torch.cpu else "cuda")

model_name = 'bert-base-uncased'

model = BertForSequenceClassification.from_pretrained(model_name, num_labels = 13)

model.to(device)
tokenizer = BertTokenizer.from_pretrained(model_name)

inputs = tokenizer(tokens,
                   padding=True,
                   truncation=True,
                   max_length= 100,
                   return_tensors="pt")

labels = torch.tensor(df_texte.label_cat1 , dtype=torch.long)

inputs.to(device)

labels.to(device)
# Entraîner le modèle



Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly i

tensor([9, 0, 0,  ..., 0, 0, 0])

In [40]:
inputs

{'input_ids': tensor([[  101,  3145,  3444,  ...,  4408,  3103,   102],
        [  101, 12827,  2938,  ...,     0,     0,     0],
        [  101,  3145,  3444,  ...,  2300,  3707,   102],
        ...,
        [  101,  4965, 19169,  ...,     0,     0,     0],
        [  101,  4965,  2813,  ...,     0,     0,     0],
        [  101,  4965, 19169,  ...,     0,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 1],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}

In [41]:

train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(inputs['input_ids'], labels, 
                                                            random_state=2018, test_size=0.2)
# Performing same steps on the attention masks
train_masks, validation_masks, _, _ = train_test_split(inputs['attention_mask'], labels,
                                             random_state=2018, test_size=0.2)

In [42]:
len(train_inputs), len(train_masks), len(train_labels), len(validation_inputs), len(validation_masks[:840]), len(validation_labels)

(840, 840, 840, 210, 210, 210)

In [43]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

#Creating the DataLoader which will help us to load data into the GPU/CPU
batch_size = 20

# Create the DataLoader for our training set.
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

# Create the DataLoader for our validation set.
validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

In [44]:
# AdamW is an optimizer which is a Adam Optimzier with weight-decay-fix
optimizer = AdamW(model.parameters(),
                  lr = 2e-5, 
                  eps = 1e-8 
                )

C:\Users\Dragomir\anaconda3\lib\site-packages\transformers\optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [45]:
from transformers import get_linear_schedule_with_warmup

# Number of training epochs (authors recommend between 2 and 4)
epochs = 2

# Total number of training steps is number of batches * number of epochs.
total_steps = len(train_dataloader) * epochs
print(len(train_dataloader))
# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)
scheduler

42


In [46]:
import numpy as np

# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return sum(pred_flat == labels_flat) / len(labels_flat)



In [47]:
#Creating the helper function to have a watch on elapsed time

import time
import datetime

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [48]:
# Store the average loss after each epoch so we can plot them.
loss_values = []
epochs = 3
for epoch in range(0,epochs):
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch + 1, epochs))
    print('Training...')
    
    t0 = time.time()
    model.train()
    total_loss = 0
    
    for step, batch in enumerate(train_dataloader):
        b_input_ids = batch[0]
        b_input_mask = batch[1]
        b_labels = batch[2]
        
        # Always clear any previously calculated gradients before performing a
        # backward pass. PyTorch doesn't do this automatically because 
        # accumulating the gradients is "convenient while training RNNs". 
        # (source: https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch)
        model.zero_grad()  
        # Progress update every 40 batches.
        if step % 10 == 0 and not step == 0:
            # Calculate elapsed time in minutes.
            elapsed = format_time(time.time() - t0)
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))
            
        optimizer.zero_grad()
        
        outputs = model(input_ids = b_input_ids ,attention_mask = b_input_mask, labels=b_labels)
        
        # The call to `model` always returns a tuple, so we need to pull the 
        # loss value out of the tuple.
        loss = outputs[0]

        # Accumulate the training loss over all of the batches so that we can
        # calculate the average loss at the end. `loss` is a Tensor containing a
        # single value; the `.item()` function just returns the Python value 
        # from the tensor.
        total_loss += loss.item()

        # Perform a backward pass to calculate the gradients.
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # Update parameters and take a step using the computed gradient.
        # The optimizer dictates the "update rule"--how the parameters are
        # modified based on their gradients, the learning rate, etc.
        optimizer.step()

        # Update the learning rate.
        scheduler.step()
        
        # Calculate the average loss over the training data.
        avg_train_loss = total_loss / len(train_dataloader)            

        # Store the loss value for plotting the learning curve.
        loss_values.append(avg_train_loss)
        if step == 40:
            print("")
            print("  Average training loss: {0:.2f}".format(avg_train_loss))
            print("  Training epoch took: {:}".format(format_time(time.time() - t0)))

            print("")
            print("Running Validation...")
        
    t0 = time.time()

    # Put the model in evaluation mode--the dropout layers behave differently
    # during evaluation.
    model.eval()

    # Tracking variables 
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0

    for batch in validation_dataloader:
        
        
        batch = tuple(t for t in batch)
        
        # Unpack the inputs from our dataloader
        b_input_ids, b_input_mask, b_labels = batch
        
        # Telling the model not to compute or store gradients, saving memory and
        # speeding up validation
        with torch.no_grad():        

            # Forward pass, calculate logit predictions.
            # This will return the logits rather than the loss because we have
            # not provided labels.
            # token_type_ids is the same as the "segment ids", which 
            # differentiates sentence 1 and 2 in 2-sentence tasks.
            # The documentation for this `model` function is here: 
            # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
            outputs = model(b_input_ids,
                            token_type_ids=None, 
                            attention_mask=b_input_mask)
        # Put the model in evaluation mode--the dropout layers behave differently
        # during evaluation.
        logits = outputs[0]
        
        # Calculate the accuracy for this batch of test sentences.
        tmp_eval_accuracy = flat_accuracy(logits, b_labels)
        
        # Accumulate the total accuracy.
        eval_accuracy += tmp_eval_accuracy

        # Track the number of batches
        nb_eval_steps += 1

    # Report the final accuracy for this validation run.
    print("  Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
    print("  Validation took: {:}".format(format_time(time.time() - t0)))


======== Epoch 1 / 3 ========
Training...
  Batch    10  of     42.    Elapsed: 0:01:17.
  Batch    20  of     42.    Elapsed: 0:02:32.
  Batch    30  of     42.    Elapsed: 0:03:50.
  Batch    40  of     42.    Elapsed: 0:05:06.

  Average training loss: 2.11
  Training epoch took: 0:05:13

Running Validation...
  Accuracy: 0.66
  Validation took: 0:00:27

======== Epoch 2 / 3 ========
Training...
  Batch    10  of     42.    Elapsed: 0:01:16.
  Batch    20  of     42.    Elapsed: 0:02:32.
  Batch    30  of     42.    Elapsed: 0:03:49.
  Batch    40  of     42.    Elapsed: 0:05:05.

  Average training loss: 1.58
  Training epoch took: 0:05:12

Running Validation...
  Accuracy: 0.69
  Validation took: 0:00:27

======== Epoch 3 / 3 ========
Training...
  Batch    10  of     42.    Elapsed: 0:01:17.
  Batch    20  of     42.    Elapsed: 0:02:33.
  Batch    30  of     42.    Elapsed: 0:03:49.
  Batch    40  of     42.    Elapsed: 0:05:06.

  Average training loss: 1.48
  Training epoch t

In [80]:
loss_values

[0.06276288486662365,
 0.12530690147763207,
 0.1868147679737636,
 0.24321325052352177,
 0.29947900204431444,
 0.35791354519980295,
 0.4151692731039865,
 0.4740180742172968,
 0.5299875793002901,
 0.588047464688619,
 0.6437131904420399,
 0.699635142371768,
 0.7501753795714605,
 0.8038823434284755,
 0.8553483202343896,
 0.9088643164861769,
 0.9607306434994652,
 1.013925035794576,
 1.0660182237625122,
 1.120287231036595,
 1.1704101789565313,
 1.2206562019529796,
 1.2714766263961792,
 1.318939467271169,
 1.3677479795047216,
 1.4150880773862202,
 1.4587522205852328,
 1.5057093132109869,
 1.5542563994725545,
 1.6015789310137432,
 1.6510472212518965,
 1.6983090184983753,
 1.7464575256620134,
 1.7906955991472517,
 1.8333004258927845,
 1.8817569812138875,
 1.9300561916260492,
 1.9740923614729018,
 2.016504148642222,
 2.063247186797006,
 2.1074296094122387,
 2.1479559938112893,
 0.045001194590613955,
 0.0875028343427749,
 0.12814861536026,
 0.17126727388018653,
 0.2110118241537185,
 0.25090584584

In [45]:
print('classe du texte à prédire'.format(labels[12]))

classe du texte à prédire


In [71]:
def eval_bert():
    model.eval()
    y_pred  = []
    # Tracking variables 
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0

    for batch in validation_dataloader:
        
        batch = tuple(t for t in batch)
        
        # Unpack the inputs from our dataloader
        b_input_ids, b_input_mask, b_labels = batch
        
        # Telling the model not to compute or store gradients, saving memory and
        # speeding up validation
        with torch.no_grad():        

            # Forward pass, calculate logit predictions.
            # This will return the logits rather than the loss because we have
            # not provided labels.
            # token_type_ids is the same as the "segment ids", which 
            # differentiates sentence 1 and 2 in 2-sentence tasks.
            # The documentation for this `model` function is here: 
            # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
            outputs = model(b_input_ids,
                            token_type_ids=None, 
                            attention_mask=b_input_mask,
                            output_hidden_states = True)
        # Put the model in evaluation mode--the dropout layers behave differently
        # during evaluation.
        logits = outputs[0]
        out = outputs
        
        #récupère les prédictions
        y_pred.append(logits)
        
        # Calculate the accuracy for this batch of test sentences.
        tmp_eval_accuracy = flat_accuracy(logits, b_labels)
        
        # Accumulate the total accuracy.
        eval_accuracy += tmp_eval_accuracy

        # Track the number of batches
        nb_eval_steps += 1

    # Report the final accuracy for this validation run.
    print("  Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
    print("  Validation took: {:}".format(format_time(time.time() - t0)))
    return y_pred,out

In [72]:
pred, out  = eval_bert()

  Accuracy: 0.69
  Validation took: 0:18:54


In [57]:
pred_labels = []
for nb, classe in enumerate(pred):
    for i in range(len(pred[nb])):
        predicted_class = torch.argmax(pred[nb][i]).item()
        pred_labels.append(predicted_class)

In [77]:
len(out.hidden_states[0][0][0])

768

In [58]:
np.array(pred_labels)

array([ 8, 12,  9, 12,  9,  0,  0, 10,  3,  1,  9, 10,  1,  8,  3,  8,  8,
        3,  0,  3, 12,  0,  9,  9,  1,  8,  8,  8,  9, 12,  8,  9,  9,  3,
        9, 10,  1,  3,  8,  8, 10,  9,  8,  8,  8,  8,  0, 12,  1,  0, 10,
        9,  9, 12,  3, 10,  1,  0,  0,  8,  9,  9,  3,  8,  1,  0,  3,  3,
       12,  3,  9,  8,  9,  9,  1,  0,  9, 10,  9, 12, 12, 10,  0, 10,  0,
        8,  0,  8,  3,  3,  8,  3,  9,  9,  3,  3,  1,  3,  3,  8, 12, 10,
        9,  9,  0,  9,  1,  3, 12,  9,  1,  8,  3, 10, 12,  1, 12,  3,  9,
        1,  3,  9,  0, 10,  1,  8,  1,  9,  3,  0,  3,  0, 12,  8,  8,  1,
       12, 10, 10,  0,  9, 10,  3,  3,  0, 12,  0,  9,  8,  8,  0,  0,  3,
        0,  0,  0,  8,  9, 12, 10, 12, 12,  8,  1,  0,  9,  3, 10,  0,  8,
        8, 12,  0,  1, 12,  9,  8,  3, 12,  8, 10,  8,  8,  0,  1,  3, 12,
       10,  9,  3,  0,  8,  3,  8,  1,  0,  9, 12,  3,  3,  9,  3,  0,  0,
        9,  0,  0,  8,  9,  1])

In [59]:
validation_labels

tensor([ 8, 12,  8, 12,  9,  9,  9,  8,  3,  1,  9, 10,  1,  8,  3,  8,  8,  3,
        10,  3, 12,  0,  9,  9,  1, 10, 10,  8,  8, 12,  8,  0,  9,  3,  9, 10,
         1,  4, 10, 10, 10,  8,  8,  8,  8, 10,  0, 12,  1,  0, 10,  9,  9, 12,
         3, 10,  1,  0,  1,  8,  8,  0,  3,  0,  1,  0,  3,  0, 12,  3,  9,  1,
         0,  9,  1,  0,  0, 10,  0, 12, 12, 10,  0, 10,  0, 10,  1,  8,  4,  4,
         8,  3,  9,  9,  3,  3,  1,  3,  4,  8, 12,  9,  9,  9,  0,  9,  1,  4,
        12,  9,  1,  8,  4,  0, 12,  1, 12,  4,  9,  1,  3,  9,  0, 10,  1, 10,
         1,  9,  4,  0,  3,  0, 12,  8,  8,  8, 12, 10, 10,  0,  9, 10,  3,  2,
         1, 12,  1,  9, 10,  8,  0,  0,  4,  9,  9,  0,  8,  9, 12, 10, 12, 12,
        10,  1,  9,  9,  3, 10,  1, 10,  8, 12,  0,  1, 12,  9, 10, 10, 12, 10,
        10,  8,  8,  0,  1,  7, 12, 12,  9,  4,  0,  8,  8,  8,  4,  9,  9, 12,
         3,  3, 10,  3,  1,  0,  0,  0,  9, 10,  9,  0])

In [60]:
print(classification_report(validation_labels,pred_labels))

              precision    recall  f1-score   support

           0       0.60      0.68      0.64        31
           1       0.86      0.72      0.78        25
           2       0.00      0.00      0.00         1
           3       0.57      1.00      0.73        20
           4       0.00      0.00      0.00        11
           7       0.00      0.00      0.00         1
           8       0.61      0.77      0.68        30
           9       0.71      0.77      0.74        35
          10       0.79      0.48      0.60        31
          12       1.00      0.96      0.98        25

    accuracy                           0.70       210
   macro avg       0.51      0.54      0.51       210
weighted avg       0.69      0.70      0.68       210



C:\Users\Dragomir\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Dragomir\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Dragomir\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### Classification de texte avec Universal sentence encoder

In [83]:
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")
embeddings = embed(tokens)

print(embeddings)

tf.Tensor(
[[-0.05380953 -0.05427526  0.03763935 ...  0.04525556  0.0400887
  -0.04485746]
 [-0.0504561  -0.04426588  0.0262354  ...  0.02325921  0.02831347
   0.03676867]
 [-0.05464859 -0.05145617 -0.02325417 ...  0.05250543 -0.03077259
  -0.04503982]
 ...
 [-0.01919353 -0.04591073 -0.02684828 ... -0.02107586  0.04736629
  -0.0543308 ]
 [-0.0112249  -0.04495183  0.03897667 ...  0.01591754 -0.01513253
  -0.00175135]
 [-0.05246769 -0.04012159  0.03473873 ...  0.02537613 -0.01057997
   0.002412  ]], shape=(1050, 512), dtype=float32)


In [84]:
embeddings

<tf.Tensor: shape=(1050, 512), dtype=float32, numpy=
array([[-0.05380953, -0.05427526,  0.03763935, ...,  0.04525556,
         0.0400887 , -0.04485746],
       [-0.0504561 , -0.04426588,  0.0262354 , ...,  0.02325921,
         0.02831347,  0.03676867],
       [-0.05464859, -0.05145617, -0.02325417, ...,  0.05250543,
        -0.03077259, -0.04503982],
       ...,
       [-0.01919353, -0.04591073, -0.02684828, ..., -0.02107586,
         0.04736629, -0.0543308 ],
       [-0.0112249 , -0.04495183,  0.03897667, ...,  0.01591754,
        -0.01513253, -0.00175135],
       [-0.05246769, -0.04012159,  0.03473873, ...,  0.02537613,
        -0.01057997,  0.002412  ]], dtype=float32)>

In [85]:
embeddings.shape
embeddings.numpy()

array([[-0.05380953, -0.05427526,  0.03763935, ...,  0.04525556,
         0.0400887 , -0.04485746],
       [-0.0504561 , -0.04426588,  0.0262354 , ...,  0.02325921,
         0.02831347,  0.03676867],
       [-0.05464859, -0.05145617, -0.02325417, ...,  0.05250543,
        -0.03077259, -0.04503982],
       ...,
       [-0.01919353, -0.04591073, -0.02684828, ..., -0.02107586,
         0.04736629, -0.0543308 ],
       [-0.0112249 , -0.04495183,  0.03897667, ...,  0.01591754,
        -0.01513253, -0.00175135],
       [-0.05246769, -0.04012159,  0.03473873, ...,  0.02537613,
        -0.01057997,  0.002412  ]], dtype=float32)

In [89]:
pipe = Pipeline([
                ("estimator",LogisticRegression())
                ])
pipe

X = embed
y = df_texte.label_cat1

param_grid = {
              'estimator':[RandomForestClassifier(),
                           AdaBoostClassifier(),
                           GradientBoostingClassifier(),
                           ExtraTreesClassifier(),
                           DecisionTreeClassifier(),
                           SVC(),
                           BaggingClassifier(),
                           KNeighborsClassifier(13),
                           LogisticRegression()
                          ]}


grid = GridSearchCV(pipe,param_grid=param_grid, cv=10,return_train_score=True ,n_jobs=-1, verbose=1)

grid.fit(document_vectors,y)

grid_cv = pd.DataFrame(grid.cv_results_)

new_cols = [i for i in grid_cv.columns if 'split' not in i.lower()]

grid_cv.loc[:,new_cols].sort_values('mean_test_score', ascending=False)



Fitting 10 folds for each of 9 candidates, totalling 90 fits


C:\Users\Dragomir\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:676: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  warnings.warn(


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_estimator,params,mean_test_score,std_test_score,rank_test_score,mean_train_score,std_train_score
3,0.546545,0.078662,0.034702,0.010478,ExtraTreesClassifier(),{'estimator': ExtraTreesClassifier()},0.837143,0.091493,1,1.000000,0.000000
0,1.079431,0.095665,0.026701,0.007825,RandomForestClassifier(),{'estimator': RandomForestClassifier()},0.826667,0.105607,2,1.000000,0.000000
5,0.094993,0.013918,0.026303,0.007667,SVC(),{'estimator': SVC()},0.824762,0.088032,3,0.868360,0.005838
8,0.158104,0.022050,0.000500,0.000500,LogisticRegression(),{'estimator': LogisticRegression()},0.824762,0.097329,3,0.853228,0.008598
2,53.011235,1.596420,0.004398,0.000663,GradientBoostingClassifier(),{'estimator': GradientBoostingClassifier()},0.796190,0.103735,5,1.000000,0.000000
6,0.826296,0.080196,0.003501,0.000500,BaggingClassifier(),{'estimator': BaggingClassifier()},0.792381,0.110065,6,0.994497,0.002495
7,0.002500,0.000502,0.009898,0.002073,KNeighborsClassifier(n_neighbors=13),{'estimator': KNeighborsClassifier(n_neighbors...,0.788571,0.107226,7,0.849524,0.007628
4,0.189835,0.027661,0.000801,0.000401,DecisionTreeClassifier(),{'estimator': DecisionTreeClassifier()},0.748571,0.104084,8,1.000000,0.000000
1,1.320115,0.222486,0.018453,0.003195,AdaBoostClassifier(),{'estimator': AdaBoostClassifier()},0.290476,0.039784,9,0.296402,0.017020


In [98]:
from sklearn.ensemble import ExtraTreesClassifier



X = embeddings.numpy()
y = df_texte.label_cat1

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)
clf = ExtraTreesClassifier(n_estimators=100, random_state=0)
clf.fit(X_train, y_train)

ExtraTreesClassifier(random_state=0)

In [99]:
y_pred = clf.predict(X_test)
accuracy_score(y_test,y_pred)

0.8952380952380953

In [101]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.88      0.74      0.81        31
           1       0.92      0.88      0.90        40
           2       0.00      0.00      0.00         1
           3       0.96      1.00      0.98        22
           4       0.88      1.00      0.93         7
           5       0.00      0.00      0.00         1
           8       0.76      1.00      0.86        22
           9       0.86      0.94      0.90        34
          10       0.92      0.81      0.86        27
          12       1.00      1.00      1.00        25

    accuracy                           0.90       210
   macro avg       0.72      0.74      0.72       210
weighted avg       0.89      0.90      0.89       210



C:\Users\Dragomir\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Dragomir\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Dragomir\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [56]:
np.sort(y_test.unique())

array([ 0,  1,  2,  3,  4,  6,  8,  9, 10, 12])

In [57]:
np.array(y_test)

array([10, 10,  9,  1,  1,  8,  3,  3,  1,  1,  8,  1,  3,  8,  1,  0,  0,
        0,  9,  9,  9,  2,  0,  3, 10, 10,  3,  0, 12,  9,  1,  9,  8,  8,
        9, 12,  3,  8,  0,  8,  0,  1,  0, 12,  0,  0, 10,  8, 10, 12, 12,
       10,  6,  3,  1, 12, 10,  1, 12,  0,  1,  4,  9,  8,  3, 10,  1, 12,
        0,  0,  9,  9,  9, 12, 12,  1,  1,  9,  3,  8,  0,  0,  9,  1, 10,
        9,  0,  3,  9,  3,  3,  8, 10,  8,  1,  3,  0,  0,  1,  9,  8,  8,
        8,  9,  0,  9, 12, 10, 10,  8,  8,  8,  9,  1,  3,  0,  1, 10,  9,
        4,  8,  1, 12, 10,  9,  6,  8, 10,  0,  1, 12, 12,  4,  0, 12, 10,
        8,  9,  9, 12,  3,  0,  8, 10,  8,  2,  0,  1,  8,  8, 12,  8,  3,
        3,  0,  0,  0,  9, 12,  8,  9,  8,  8,  4,  8, 12,  0,  1, 12, 10,
        8, 10, 12,  9,  0,  0,  3, 12, 12, 10,  1,  9,  1, 10, 10,  8,  1,
        9,  0, 12,  8, 10,  9, 10,  3,  1,  0,  1, 12,  0, 10,  4, 12, 12,
        0, 10,  0, 12,  8,  9])

In [58]:
pd.Series(y_pred).unique()

array([10,  9,  1,  3,  8,  0, 12,  4])